# 18. CNN with CIFAR10

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
from torch.utils.data import DataLoader

import torchvision.utils
import torchvision.datasets as dsets
import torchvision.transforms as transforms

import numpy as np
import os

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline

## 18.1 Prepare Data

In [3]:
transform = transforms.Compose([
    # Data Augmentation
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.Resize(32),
    
    # Data Nomalization
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))
])

# Normalize a tensor image with mean and standard deviation.
# Given mean: (M1,...,Mn) and std: (S1,..,Sn) for n channels,
# this transform will normalize each channel of the input torch.
# *Tensor i.e. input[channel] = (input[channel] - mean[channel]) / std[channel]

train_data = dsets.CIFAR10(root='./data', 
                           train=True,
                           download=True, 
                           transform=transform)

test_data  = dsets.CIFAR10(root='./data', 
                           train=False,
                           download=True, 
                           transform=transform)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
batch_size = 128

train_loader = DataLoader(train_data, 
                          batch_size=batch_size,
                          shuffle=True)

test_loader = DataLoader(test_data, 
                         batch_size=batch_size,
                         shuffle=False)

## 18.2 Define Model

In [7]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        
        self.conv_layer = nn.Sequential(
            nn.Conv2d(3, 32, 5),
            
            # Batch Nomalization
            nn.BatchNorm2d(32),
            
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            nn.Conv2d(32, 64, 5),
            
            # Batch Nomalization
            nn.BatchNorm2d(64),
            
            nn.ReLU(),
            nn.MaxPool2d(2,2)
        )

        self.fc_layer = nn.Sequential(
            nn.Linear(64 * 5 * 5, 100),
            # Dropout
            # nn.Dropout(0.5)
            nn.ReLU(),
            nn.Linear(100, 10)              
        )
        
        # Weight Initialization
        for m in self.modules() :
            if isinstance(m, nn.Conv2d):
                # init.xavier_normal(m.weight.data)
                init.kaiming_normal_(m.weight.data)
                m.bias.data.fill_(0)      
            if isinstance(m, nn.Linear):
                # init.xavier_normal(m.weight.data)
                init.kaiming_normal_(m.weight.data)
                m.bias.data.fill_(0)                
        
    def forward(self, x):
        out = self.conv_layer(x)
        out = out.view(-1, 64*5*5)
        out = self.fc_layer(out)
        
        return out
    
model = CNN().cuda()

In [8]:
loss = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Momentum & Weight Regularization(L2)
# optim.SGD(model.parameters(), lr=1e-2, momentum=0.9, weight_decay=1e-5)

## 18.3 Train Model

In [9]:
num_epochs = 10

In [10]:
for epoch in range(num_epochs):

    total_batch = len(train_data) // batch_size
    
    for i, (batch_images, batch_labels) in enumerate(train_loader):
        
        X = batch_images.cuda()
        Y = batch_labels.cuda()

        pre = model(X)
        cost = loss(pre, Y)

        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        if (i+1) % 200 == 0:
            print('Epoch [%d/%d], lter [%d/%d], Loss: %.4f'
                 %(epoch+1, num_epochs, i+1, total_batch, cost.item()))

Epoch [1/10], lter [200/390], Loss: 1.3217
Epoch [2/10], lter [200/390], Loss: 1.0757
Epoch [3/10], lter [200/390], Loss: 1.0477
Epoch [4/10], lter [200/390], Loss: 0.7904
Epoch [5/10], lter [200/390], Loss: 0.7576
Epoch [6/10], lter [200/390], Loss: 0.7350
Epoch [7/10], lter [200/390], Loss: 0.8679
Epoch [8/10], lter [200/390], Loss: 0.6426
Epoch [9/10], lter [200/390], Loss: 0.8302
Epoch [10/10], lter [200/390], Loss: 0.6421


## 18.4 Test Model

In [11]:
# 이제부터는 꼭 해주어야함.
# Batch Norm과 Dropout은 Train과 Test일 때 서로 다르게 행동함.
model.eval()

# 반대로 다시 Train을 해야하면
# model.train()

correct = 0
total = 0

for images, labels in test_loader:
    
    images = images.cuda()
    outputs = model(images)
    
    _, predicted = torch.max(outputs.data, 1)
    
    total += labels.size(0)
    correct += (predicted == labels.cuda()).sum()
    
print('Accuracy of test images: %f %%' % (100 * float(correct) / total))

Accuracy of test images: 72.580000 %


참고로 지난 번 수업(4주차, 14번 코드)에서는 정확도 **47.960000%**를 기록